<a href="https://colab.research.google.com/github/vimesh630/Revenue_Forecasting/blob/main/Dataset_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [4]:
import pandas as pd

# Mount Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Define File Paths

In [6]:
input_path = "/content/drive/MyDrive/VERGER/Revenue_Forecasting/forecasting_data.csv"
output_path = "/content/drive/MyDrive/VERGER/Revenue_Forecasting/forecasting_data_processed.csv"

df=pd.read_csv(input_path)

df.columns=df.columns.str.strip()

# Create Peiod Column

In [7]:
df["Period"] = pd.to_datetime(
    df["Year"].astype(str) + "-" + df["Month_No"].astype(str).str.zfill(2)
)

/tmp/ipython-input-2335711947.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Period"] = pd.to_datetime(


DateParseError: Unknown datetime string format, unable to parse: 2018.0-8.0, at position 0

# Sort the Data

In [ ]:
df = df.sort_values(["Account", "Product", "Period"]).reset_index(drop=True)

# Create Lag Features

In [ ]:
lags = [1, 3, 6]  # you can adjust these
for lag in lags:
    df[f"Revenue_Lag_{lag}"] = df.groupby(["Account", "Product"])["Revenue"].shift(lag)
    df[f"Quantity_Lag_{lag}"] = df.groupby(["Account", "Product"])["Quantity"].shift(lag)

# Create Rolling Features

In [ ]:
windows = [3, 6]  # number of months
for window in windows:
    df[f"Revenue_Rolling_{window}"] = df.groupby(["Account", "Product"])["Revenue"].transform(lambda x: x.rolling(window).mean())
    df[f"Quantity_Rolling_{window}"] = df.groupby(["Account", "Product"])["Quantity"].transform(lambda x: x.rolling(window).mean())

# Save the File

In [ ]:
df.to_csv(output_path, index=False)

print(f"✅ Processed dataset saved to: {output_path}")
print(df.head(10))